#### Standard Bloom Filter

We implement a standard standard bloom filter which is a 1-d bit array `B` of size `m` and we use `k` hash functions, each of which maps integer keys $U = \set{0,1,2,3,...,|U|-1} \to \set{0,1,...,m-1}$.

In [23]:
import sympy as sp
import random
random.seed(123)

In [49]:
class TwoUniversalHashFamily:
    def __init__(self, m, max_key):
        self.m = m  # Size of the hash table
        self.p = sp.nextprime(max_key)  # generate a large prime number, greater than any key
        self.a = random.randint(1, self.p-1)  # Choose a randomly
        self.b = random.randint(0, self.p-1)  # Choose b randomly

    def hash(self, k):
        return ((self.a * k + self.b) % self.p) % self.m
    
    def __call__(self, k):
        return self.hash(k)


class BloomStandard:
    def __init__(self, m, k=None, max_key=10000000):
        self.m = m
        if k is None:
            self.k = int((m/max_key) * sp.log(2))  # optimal number of hash functions for a given m and n       
        else:
            self.k = k
        # draw k random hash functions from universal hash family
        self.h = [TwoUniversalHashFamily(m, max_key) for _ in range(k)]
        self.B = [0] * m

    def construct(self, S):
        for key in S:
            self.insert(key)
        print(f"Bloom filter constructed! Size: {self.m}, Number of hash functions: {self.k}")

    # insert new integer key into the bloom filter 
    def insert(self, key):
        for i in range(self.k):
            self.B[self.h[i](key)] = 1

    # poerform membership query for the given key
    def query(self, key):
        q = [0]*self.k
        for i in range(self.k):
            q[i] = self.B[self.h[i](key)]
        if 0 in q:
            return False
        else:
            return True    
            
    def __str__(self):
        return str(self.B)

In [50]:
# test the bloom filter
S = {5, 7, 11, 13, 17, 23, 29, 31, 37, 41, 43, 47}
m = 20
k = 3

bf = BloomStandard(m, k)
bf.construct(S)
print(bf)

Bloom filter constructed! Size: 20, Number of hash functions: 3
[1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1]


In [51]:
# membership query testing
q_size = 10

# generate a random query stream which contains keys from 0 to 1000 and does not contain keys from S
Q = set(range(100)) - S
Q = random.sample(list(Q), q_size)

print(f"Query stream: {Q}")

num_FP = 0
for key in Q:
    found = bf.query(key)
    if found:
        num_FP += 1
        print(f"Key {key} is in S: {found} --> False Positive!")
    else:
        print(f"Key {key} is not in S: {found}")

Query stream: [20, 2, 66, 96, 75, 21, 73, 80, 6, 72]
Key 20 is not in S: False
Key 2 is not in S: False
Key 66 is in S: True --> False Positive!
Key 96 is in S: True --> False Positive!
Key 75 is not in S: False
Key 21 is not in S: False
Key 73 is not in S: False
Key 80 is not in S: False
Key 6 is not in S: False
Key 72 is in S: True --> False Positive!
